In [1]:
import sys
import os
# Add parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
        
from mha021 import *

In [ ]:
# Input data:
k = 1 # W/m
t = 1 # m
Q = 2 # W/m^3
T_bottom = 3 # deg C

D = k* np.array([
    [1, 0],
    [0, 1]
])  # Constitutive matrix

mesh = MeshGenerator.structured_rectangle_mesh(
        width=1, 
        height=2, 
        nx=2,
        ny=4,
        element_type="tri", 
        dofs_per_node=1
    )
fig = mesh.plot(title=f"Mesh")
fig.show()

## Create and assemble element matrices

# System matrices
num_nodes = mesh.nodes.shape[0]
num_el = mesh.elements.shape[0]
num_dofs = num_nodes 
K = np.zeros((num_dofs, num_dofs))
f = np.zeros((num_dofs, 1))
for el in range(num_el):
    el_nodes = mesh.nodes[mesh.elements[el]-1]
    Ke, fe = flow2t_Ke_fe(el_nodes, D=D, t=t, Q=Q) # in mtm021.py 
    el_dofs = mesh.edofs[el, :]
    assem(K, Ke, el_dofs)
    assem(f, fe, el_dofs)


# Solve the system of equations
bottom_dofs = mesh.edges["bottom"]

bc_dofs = bottom_dofs
bc_vals = np.ones_like(bc_dofs)*T_bottom


# Solve system
a, r = solve_eq(K, f, bc_dofs, bc_vals)

# Plot temperature field
Ed = extract_dofs(a, mesh.edofs)
fig = plot_scalar_field(mesh.nodes, mesh.elements, Ed, title="Temperature [ᴼC]")
fig.show()

## Postprocessing

In [16]:
# Heat flux vector
q = np.zeros((num_el, 2))
for el in range(num_el):
    el_nodes = mesh.nodes[mesh.elements[el]-1]
    qe = flow2t_qe(el_nodes, D, Ed[el, :]) # in mtm021.py 
    q[el, :] = qe  
    
plot_vector_field(mesh.nodes, mesh.elements, q, title="Heat flux")

In [18]:
## Convection

# Convection Assume we have convection on the top part. We then assemble convective contributions 
# alpha = 5
# T0 = 15 
# conv_nodes = mesh.edges["top"]
# num_edges = len(conv_nodes)-1 # one less tan number of nodes
# for edge in range(num_edges):
#     edge_nodes = conv_nodes[edge:edge+2] - 1
#     nodes = mesh.nodes[edge_nodes, :]
#     # displayvar("nodes", nodes)
#     Kec, fec = convection_Ke_fe(nodes, alpha, t, T0)
#     dofs = edge_nodes + 1 # 1-index
#     # displayvar("edge dofs", dofs)
#     assem(K, Kec, dofs)
#     assem(f, fec, dofs)